In [2]:
# import sys
# print(sys.executable)

C:\Users\Admin\Jupyter\Posture Detection System\posture_env\Scripts\python.exe


In [ ]:
# !pip install scikit-learn pandas 
# !pip install mediapipe opencv-python

### Setup Mediapipe

In [3]:
import mediapipe as mp
import cv2
print("Mediapipe is working! Version:", mp.__version__)
print("Computer Vision is working! Version:", cv2.__version__)

Mediapipe is working! Version: 0.10.21
Computer Vision is working! Version: 4.11.0


In [13]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Get Realtime Webcam Feed

In [14]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    if not ret: 
        break
        
    cv2.imshow("Raw Webcam Feed",frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

### Make Detections from Feed

In [15]:
cap = cv2.VideoCapture(0)

# Initialize pose model
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make detections
    results = pose.process(image)

    # Recolor back to BGR for rendering 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Pose Detection', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
